# ParFuMor (version YAML & Objets)

In [1]:
# -*- coding: utf8 -*-
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [2]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/"
numerosKalabas=[1,2,3,4,5]
#numerosKalabas=[1]
nomKalabas=[repertoire+"16-K%d/"%num for num in numerosKalabas]

lexiqueTestPrep=u"dans"
lexiqueTestNoun=u"loups"
lexiqueTestHyper=u"villagEOIs"

In [3]:
PFM.duplicateErrors=[]
for serie in nomKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K1/
ADJ : Genre Nombre
NOM : Genre Nombre
DET : Genre Nombre
VER : Temps Pers
head stems
head stems,NOM
head stems,VER

/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K2/
ADJ : Genre Nombre
NOM : Genre Nombre Cas
DET : Genre Nombre Cas
VER : Temps Pers Genre
head stems
head stems,NOM
head stems,NOM,N
head stems,NOM,M
head stems,NOM,F
head stems,DET
head stems,VER

/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K3/
ADJ : Genre Nombre
NOM : Genre Nombre Cas
DET : Genre Nombre Cas
VER : Trans Temps Pers Genre Nombre
head stems
head stems,ADJ
head stems,NOM
head stems,NOM,A
head stems,NOM,C
head stems,NOM,B
head stems,NOM,D
head stems,DET
head stems,VER
head stems,VER,V1
head stems,VER,V2

/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K4/
ADJ : Genre Nombre
NOM : Genre Nombre Cas
DET : Nombre Cas
VER : Trans Tem

ParFuMor.py:412: UserWarning: Vedettes homographes "enfant" pour "enfant.A.N2" et "enfant.B.N3"
  warnings.warn('Vedettes homographes "%s" pour "%s" et "%s"'%(formes[0],self.vedettes[formes[0]],nom))


head stems,VER
head stems,VER,V1
head stems,VER,V2

/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K5/
ADJ : Genre Nombre
NOM : Genre Nombre Cas
DET : Nombre Cas
VER : Trans Temps Pers Genre Nombre
head stems
head stems,ADJ
head stems,NOM
head stems,NOM,N1
head stems,NOM,N1,N12
head stems,NOM,N1,N11
head stems,NOM,N2
head stems,NOM,N2,N22
head stems,NOM,N2,N21
head stems,VER
head stems,VER,VI
head stems,VER,VT
head stems,VER,VD


In [4]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[u"enfANT"][0]

villageoise.N2.N21.D
enfant.hyper.N1.N12.B


In [5]:
PFM.lexique.getLexemes(u"sorT")

[]

In [6]:
for element in PFM.lexique.vedettes:
    print element

coup
corps
disparaître
rouge
transformer
noir
planter
dévorer
différent
main
courir
vers
ENTRE
de
voir
villageois
tomber
blessé
cri
blessure
furieux
chef
profond
DEM
rejoindre
croc
DEF
homme
arbre
apporter
transpercer
jeter
courageux
pousser
sur
nuit
gorge
à
être
loup
terrible
souffrance
protéger
IND
forêt
fuir
Kaleb
couteau
approcher
lune
village
blanc
passer
avec
parler
après
mort
milieu
lueur
combattant
sort
enfant.hyper
sortir
louve
prendre
villageoise
devant
envahir
dans
démon
derrière
ombre
enfant
visage
serpent
chercher
petit
Mahira
soeur
Violette
suivant
colère
recouvrir
maison
grand
lance
combat
effrayé
sauter
fille
chasser
donner
aller
